In [70]:
import numpy as np
import pandas as pd
import sklearn

In [71]:
data=pd.read_csv('heart.csv')

In [72]:
data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [99]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Sex=le.fit_transform(data['Sex'])
data['Sex']=Sex

ChestPainType=le.fit_transform(data['ChestPainType'])
data['ChestPainType']=ChestPainType

RestingECG=le.fit_transform(data['RestingECG'])
data['RestingECG']=RestingECG

ExerciseAngina=le.fit_transform(data['ExerciseAngina'])
data['ExerciseAngina']=ExerciseAngina

ST_Slope=le.fit_transform(data['ST_Slope'])
data['ST_Slope']=ST_Slope

In [100]:
data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,3,110,264,0,1,132,0,1.2,1,1
914,68,1,0,144,193,1,1,141,0,3.4,1,1
915,57,1,0,130,131,0,1,115,1,1.2,1,1
916,57,0,1,130,236,0,0,174,0,0.0,1,1


In [101]:
data.isnull().any()

Age               False
Sex               False
ChestPainType     False
RestingBP         False
Cholesterol       False
FastingBS         False
RestingECG        False
MaxHR             False
ExerciseAngina    False
Oldpeak           False
ST_Slope          False
HeartDisease      False
dtype: bool

In [102]:
data.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [103]:
from sklearn.model_selection import train_test_split
x=data.drop('HeartDisease',axis=1)
y=data['HeartDisease']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=1,train_size=.75)

In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

In [105]:
randforestparams={'n_estimators':[10,20,30,40,50]}
model_details={   
                'linear'   :{'model':LinearRegression(),'params':{}},
                'rf'       :{'model':RandomForestClassifier(),'params':randforestparams},
                'dtc'      :{'model':DecisionTreeClassifier(),'params':{}},

             }

In [106]:
from sklearn.model_selection import GridSearchCV
def get_best(x,y):
    result=[]
    for model_name in model_details:
        model_dict=model_details.get(model_name)
        model=model_dict.get('model')
        params=model_dict.get('params')
        cv=GridSearchCV(model,params,return_train_score=True,cv=5)
        cv.fit(x,y)
        bp=cv.best_params_
        bs=cv.best_score_
        test_res={'model':model_name,'best_params':bp,'best_score':bs}
        result.append(test_res)
    res_df=pd.DataFrame(result)
    best_model=res_df[res_df['best_score']==res_df['best_score'].max()]
    return res_df,best_model

In [107]:
res_df,best_model=get_best(x,y)

In [108]:
res_df

,model,best_params,best_score
0,linear,{},0.413398
1,rf,{'n_estimators': 40},0.834373
2,dtc,{},0.738483


In [109]:
best_model

,model,best_params,best_score
1,rf,{'n_estimators': 40},0.834373


In [110]:
model=RandomForestClassifier()
model.fit(xtrain,ytrain)
model.score(xtest,ytest)

0.8782608695652174